In [1]:
from sentence_transformers import SentenceTransformer

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from tqdm.auto import tqdm
import numpy as np
import pandas as pd

In [3]:
model_name = 'multi-qa-distilbert-cos-v1'

In [4]:
embedding_model = SentenceTransformer(model_name)

## Q1

In [5]:
user_question = "I just discovered the course. Can I still join it?"

In [6]:
response = embedding_model.encode(user_question)

In [7]:
v = response

In [8]:
dimensions = len(response)

In [9]:
dimensions

768

In [10]:
response[0]

0.078222655

## Q2

In [11]:
import requests 

base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [12]:
type(documents)

list

In [13]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [14]:
documents[0]['course']

'data-engineering-zoomcamp'

In [15]:
def filter_document(documents):
    new_documents = documents
    documents_list = []
    for i in range(len(new_documents)):
        if (new_documents[i]['course'] == 'machine-learning-zoomcamp'):
            documents_list.append(new_documents[i])
    return documents_list
    

In [16]:
new_documents = filter_document(documents)
len(new_documents)

375

In [17]:
def create_embeddings(model, documents):
    embeddings = []
    for doc in tqdm(documents):
        question = doc['question']
        text = doc['text']
        qa_text = f'{question} {text}'
        encoded = model.encode(qa_text)
        embeddings.append(encoded)
    X = np.array(embeddings)
    return embeddings, X

In [18]:
embeddings, X = create_embeddings(embedding_model, new_documents)

100%|█████████████████████| 375/375 [01:04<00:00,  5.81it/s]


In [19]:
X.shape

(375, 768)

## Q3

In [20]:
scores = X.dot(v)

In [21]:
scores.argmax()

14

In [22]:
scores[scores.argmax()]

0.6506573

## Q4

In [23]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores)[:num_results]
        return [self.documents[i] for i in idx]


In [24]:
search_engine = VectorSearchEngine(documents=new_documents, embeddings=X)
search_engine.search(v, num_results=5)

[{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'section': 'General course-related questions',
  'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'id': 'ee58a693'},
 {'text': 'Welcome to the course! Go to the course page (http://mlzoomcamp.com/), scroll down and start going through the course materials. Then read everything in the cohort folder for your cohort’s year.\nClick on the links and start watching the videos. Also watch office hours from previous cohorts. Go to DTC youtube channel and click on Playlists and search for {course yyyy}. ML Zoomcamp was first launched in 2021.\nOr you c

In [25]:
results = search_engine.search(v, num_results=5)

In [26]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [27]:

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [28]:
def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

In [29]:

def evaluate(ground_truth, search_function, model):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        v = model.encode(q['question'])
        results = search_function(v)
        # results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        # print(relevance)
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

In [30]:
evaluate(ground_truth, search_engine.search, embedding_model)

100%|█████████████████| 1830/1830 [01:29<00:00, 20.53it/s]


{'hit_rate': 0.9579234972677596, 'mrr': 0.8541597276433341}

In [31]:
ground_truth[1]

{'question': 'Can you provide a link to sign up?',
 'course': 'machine-learning-zoomcamp',
 'document': '0227b872'}

In [32]:
results[0]

{'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
 'section': 'General course-related questions',
 'question': 'The course has already started. Can I still join it?',
 'course': 'machine-learning-zoomcamp',
 'id': 'ee58a693'}

## Q5

In [33]:
from elasticsearch import Elasticsearch

In [43]:
es_client = Elasticsearch('http://localhost:9200', request_timeout=30, max_retries=10, retry_on_timeout=True) 

In [44]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "question_text_vector": {
                "type": "dense_vector",
                "dims": 768,
                "index": True,
                "similarity": "cosine"
            }
        }
    }
}


In [45]:
index_name = 'course_questions'

In [49]:
es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [course_questions/wTFGjQxNQ0uhk5chmh4DUQ] already exists')

In [48]:
for i,doc in tqdm(enumerate(new_documents)):
    doc['question_text_vector'] = embeddings[i]
    es_client.index(index=index_name, document=doc)

0it [00:06, ?it/s]

KeyboardInterrupt

